In [ ]:
import cv2
import numpy as np

def load_and_preprocess_image(img):
    """
    Load an image, apply Gaussian blur, and convert it to HSV color space.
    
    Parameters:
        image_path (str): Path to the image file.
        
    Returns:
        hsv (numpy.ndarray): HSV-converted image.
        img (numpy.ndarray): Original blurred image.
    """
    
    img = cv2.GaussianBlur(img, (7, 7), 0)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return hsv, img

def create_color_masks(hsv):
    """
    Create binary masks for red and green colors in the image.
    
    Parameters:
        hsv (numpy.ndarray): HSV-converted image.
        
    Returns:
        mask_red (numpy.ndarray): Mask for red color.
        mask_green (numpy.ndarray): Mask for green color.
    """
    # Define red color range
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    
    # Define green color range
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([80, 255, 255])
    
    # Create masks for red and green
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    
    return mask_red, mask_green

def apply_morphological_operations(mask_red, mask_green):
    """
    Apply morphological operations to the masks to remove noise.
    
    Parameters:
        mask_red (numpy.ndarray): Mask for red color.
        mask_green (numpy.ndarray): Mask for green color.
        
    Returns:
        mask_red (numpy.ndarray): Cleaned mask for red color.
        mask_green (numpy.ndarray): Cleaned mask for green color.
    """
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, None)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, None)
    
    return mask_red, mask_green

def segment_apples(img, mask_red, mask_green):
    """
    Segment the apples using the color masks.
    
    Parameters:
        img (numpy.ndarray): Original blurred image.
        mask_red (numpy.ndarray): Mask for red color.
        mask_green (numpy.ndarray): Mask for green color.
        
    Returns:
        red_apples (numpy.ndarray): Image of red apples segmented.
        green_apples (numpy.ndarray): Image of green apples segmented.
        segmented_apples (numpy.ndarray): Image with both red and green apples segmented.
    """
    red_apples = cv2.bitwise_and(img, img, mask=mask_red)
    green_apples = cv2.bitwise_and(img, img, mask=mask_green)
    combined_mask = cv2.bitwise_or(mask_red, mask_green)
    segmented_apples = cv2.bitwise_and(img, img, mask=combined_mask)
    
    return red_apples, green_apples, segmented_apples

def find_and_count_contours(mask_red, mask_green, min_area=500):
    """
    Find and count the contours in the masks, filtering by minimum area.
    
    Parameters:
        mask_red (numpy.ndarray): Mask for red color.
        mask_green (numpy.ndarray): Mask for green color.
        min_area (int): Minimum area threshold to filter small contours.
        
    Returns:
        contours_red (list): List of contours for red apples.
        contours_green (list): List of contours for green apples.
        number_of_red_apples (int): Number of red apples.
        number_of_green_apples (int): Number of green apples.
    """
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by area
    contours_red = [cnt for cnt in contours_red if cv2.contourArea(cnt) > min_area]
    contours_green = [cnt for cnt in contours_green if cv2.contourArea(cnt) > min_area]
    
    number_of_red_apples = len(contours_red)
    number_of_green_apples = len(contours_green)
    
    return contours_red, contours_green, number_of_red_apples, number_of_green_apples

def draw_contours_and_display(img, contours_red, contours_green):
    """
    Draw the contours on the image and display the results.
    
    Parameters:
        img (numpy.ndarray): Original blurred image.
        contours_red (list): List of contours for red apples.
        contours_green (list): List of contours for green apples.
    """
    # Draw contours on the image
    cv2.drawContours(img, contours_red, -1, (0, 0, 255), 2)  # Red contours
    cv2.drawContours(img, contours_green, -1, (0, 255, 0), 2)  # Green contours
    
    # Display the image with contours
    cv2.imshow('Segmented Apples with Contours', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

for i in range(1,6):
    img = cv2.imread(cv2.samples.findFile(f"apples/apple{i}.jpg"))
    # Load and preprocess the image
    hsv, img = load_and_preprocess_image(img)

    # Create color masks
    mask_red, mask_green = create_color_masks(hsv)

    # Apply morphological operations to clean the masks
    mask_red, mask_green = apply_morphological_operations(mask_red, mask_green)

    # Segment the apples
    red_apples, green_apples, segmented_apples = segment_apples(img, mask_red, mask_green)

    # Display the segmented apples
    cv2.imshow('Original Image', img)
    cv2.imshow('Red Apples', red_apples)
    cv2.imshow('Green Apples', green_apples)
    cv2.imshow('Segmented Apples', segmented_apples)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Find and count the contours (apples)
    contours_red, contours_green, number_of_red_apples, number_of_green_apples = find_and_count_contours(mask_red, mask_green)

    # Print the number of apples
    print(f"Apple{i}")
    print(f"Number of red apples: {number_of_red_apples}")
    print(f"Number of green apples: {number_of_green_apples}")
    print()

    # Draw contours and display results
    draw_contours_and_display(img, contours_red, contours_green)





